<a href="https://colab.research.google.com/github/AlinaSz/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p') 
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d=%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train, y_train)),
            callbacks = [tensorboard_callback]
            )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(0.3),
 
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  Flatten(),

  Dense(1024, activation = 'relu' ),
  Dropout(0.3),

  Dense(1024, activation = 'relu' ),
  Dropout(0.3),

  Dense(num_classes, activation = 'softmax' )  ])



In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 47ms/step - loss: 2.5700 - accuracy: 0.2938 - val_loss: 1.1567 - val_accuracy: 0.6714
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 0.8743 - accuracy: 0.7190 - val_loss: 0.2484 - val_accuracy: 0.9209
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.3988 - accuracy: 0.8740 - val_loss: 0.1013 - val_accuracy: 0.9676
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2585 - accuracy: 0.9210 - val_loss: 0.0519 - val_accuracy: 0.9853
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.1865 - accuracy: 0.9439 - val_loss: 0.0412 - val_accuracy: 0.9872


0.9589569160997733

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.2259 - accuracy: 0.9590


0.9589568972587585

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),
 
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),
  
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),
  Flatten(),

  Dense(1024, activation = 'relu' ),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation = 'relu' ),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation = 'softmax' )  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

  model.fit(X_train, y_train,
            batch_size=int(params.get('batch_size', 128)),
            epochs = 5,
            verbose = 0,
            )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [29]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4445240856343778, 'dropout_cnn_block_three': 0.3977332190869904, 'dropout_cnn_block_two': 0.37721105512383185, 'dropout_dense_block_one': 0.3405756177029631, 'dropout_dense_block_two': 0.6946579255911409}
accuracy=0.8900226950645447
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.31972966834707744, 'dropout_cnn_block_three': 0.38157026493585955, 'dropout_cnn_block_two': 0.4547699913661386, 'dropout_dense_block_one': 0.6373351362471514, 'dropout_dense_block_two': 0.4884017475282456}
accuracy=0.8755102157592773
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.49214724713661223, 'dropout_cnn_block_three': 0.36820409733159354, 'dropout_cnn_block_two': 0.48628092816671026, 'dropout_dense_block_one': 0.3422673029193197, 'dropout_dense_block_two': 0.585363483864481}
accuracy=0.8625850081443787
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4033086890831772, 'dropout_cnn_block_three': 0.32552136448123087, 'dropout_cnn_block_two': 0.4878899194